In [1]:
import os
import torch
import torch.nn as nn
from torch.nn import functional as F
import mmap
import random
import pickle

In [2]:
torch.cuda.is_available()

True

In [3]:
device= 'cuda' if torch.cuda.is_available() else 'cpu'

block_size= 128
batch_size= 16
max_iters= 300
learning_rate= 2e-3
eval_iters= 250
n_embd= 384 #n_embd should be a very large vector, as it is intended to capture the semantics. 
dropout= 0.2 #20 percent of neurons, to prevent overfitting
n_layer= 4
n_head= 4

In [4]:
device

'cuda'

In [5]:
chars= ""

with open(r"C:\Users\amogh\openwebtext\vocab.txt", 'r', encoding= 'utf-8') as f:
    text= f.read()

chars= sorted(set(text))
vocab_size= len(chars)
vocab_size

32172

In [6]:
string_to_int= {ch:i for i, ch in enumerate(chars) }
int_to_string= {i:ch for i, ch in enumerate(chars)}
encode= lambda s: [string_to_int[c] for c in s]
decode= lambda l: ''.join(int_to_string[i] for i in l)
#simple mapping for a small vocabulary

In [7]:
def get_random_chunk(split):
    filename = r"C:\Users\amogh\openwebtext\output_train.txt" if split == 'train' else r"C:\Users\amogh\openwebtext\output_val.txt"
    #filename = r"openwebtext\output_train.txt" if split == 'train' else r"amogh\openwebtext\output_val.txt
    with open(filename, 'rb') as f:
        with mmap.mmap(f.fileno(), 0, access=mmap.ACCESS_READ) as mm:
            # Determine the file size and a random position to start reading
            file_size = len(mm)
            start_pos = random.randint(0, (file_size) - block_size*batch_size)

            # Seek to the random position and read the block of text
            mm.seek(start_pos)
            block = mm.read(block_size*batch_size-1)

            # Decode the block to a string, ignoring any invalid byte sequences
            decoded_block = block.decode('utf-8', errors='ignore').replace('\r', '')
            
            # Train and test splits
            data = torch.tensor(encode(decoded_block), dtype=torch.long)
            
    return data




In [8]:
def get_batch(split):
    data = get_random_chunk(split)
    
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

In [9]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

The skeleton of our GPT model is the Bigram built previously. 

In [11]:
class Head(nn.Module): #This shall implement the scaled dot product attention
    
    def __init__(self, head_size):
        super().__init__()
        self.key= nn.Linear(n_embd, head_size, bias= False)
        self.query= nn.Linear(n_embd, head_size, bias= False)
        self.value= nn.Linear(n_embd, head_size, bias= False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout= nn.Dropout(dropout)
        
    def forward(self, x):
            B, T, C= x.shape
            k= self.key(x)
            q= self.query(x)
            
            # compute attention scores ("affinities")
            wei = q @ k.transpose(-2,-1) * k.shape[-1]**-0.5 # (B, T, hs) @ (B, hs, T) -> (B, T, T)
            wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
            wei = F.softmax(wei, dim=-1) # (B, T, T)
            wei = self.dropout(wei)
            # perform the weighted aggregation of the values
            v = self.value(x) # (B,T,hs)
            out = wei @ v # (B, T, T) @ (B, T, hs) -> (B, T, hs)
            
            return out
            
            
            
class MultiHeadAttention(nn.Module):
    
    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads= nn.ModuleList([Head(head_size) for _ in range(num_heads)]) #n_heads running in parallel,
        self.proj= nn.Linear(head_size * num_heads, n_embd) #projecting the output of each head into the larger scale
        self.dropout= nn.Dropout(dropout)
    
    def forward(self, x):
        out= torch.cat([h(x) for h in self.heads], dim= -1) #concatenating the heads along the 
        #last dimension (Channel) in Batch, Time, Channel
        out= self.dropout(self.proj(out))
        return out



class FeedForward(nn.Module):
    
    def __init__(self, n_embd):
        super().__init__()
        self.net= nn.Sequential (
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout)
        )
    
    def forward(self, x):
        return self.net(x)


class Block(nn.Module): #for defining the decoder blocks
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head #Number of embedded features captured by each head
        self.sa = MultiHeadAttention(n_head, head_size) #Self Attention-> Multi-head attention
        self.ffwd = FeedForward(n_embd) 
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x): #Forward Pass
        y = self.sa(x) 
        x = self.ln1(x + y)
        y = self.ffwd(x)
        x = self.ln2(x + y)
        return x


class GPTLanguageModel(nn.Module):
    #Define the Embedding table
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table= nn.Embedding(vocab_size, n_embd)
        #We have to create a function that performs positional encoding to the tokenised inputs.
        self.positional_embedding_table= nn.Embedding(block_size, n_embd) #n_embd shall also be used in token encoding
        
        #for the decoder layers, we can use the common Sequential NN model with the heads corresponding to Multihead attention
        self.blocks= nn.Sequential(*[Block(n_embd, n_head= n_head) for _ in range(n_layer)]) #No.of decoder blocks running sequentially
        #* asterisk is used to imply 'For All'
        self.ln_f= nn.LayerNorm(n_embd)#final layer's layer norm
        self.ln_head= nn.LayerNorm(n_embd, vocab_size) #first layer's layer norm
        
        self.apply(self._init_weights)

    
    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean= 0.0, std= 0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean= 0.0, std= 0.02)
    
    
    #Define the forward pass for the network
    def forward(self, index, targets= None):
        
        B, T= index.shape
        
        
        #index and targets are B,T,C dimensioned tensors
        tok_emb= self.token_embedding_table(index) #B, T
        pos_emb= self.positional_embedding_table(torch.arange(T, device= device)) #T, C
        
        x= tok_emb + pos_emb  #B, T, C
        x= self.blocks(x) #here the embeddings go through the blocks, B T C
        x= self.ln_f(x) #this is the final layer
        logits=self.ln_head(x)

        
        if targets is None:
            loss= None
        else:
            B, T, C= logits.shape
        #B is for Batches, T is for Targets, C is for 
            logits= logits.view(B*T,C)
            targets= targets.view(B*T)
            loss= F.cross_entropy(logits, targets)
        
        return logits, loss
    
    
            
    
    def generate(self, index, max_new_tokens):
        #index is (B,T) array of indices in current context (context as in context of tokens in the bigram) 
        for _ in range (max_new_tokens):
            index_cond = index[:, -block_size:] # condense index to the last block_size tokens
            
            logits,loss= self.forward(index_cond) #gets predictions
            logits= logits[:,-1,:] #focussing on the last time step- property of the bigram, becomes (B,C)  
            probs= F.softmax(logits, dim= -1) # (B,C)
            
            index_next= torch.multinomial(probs, num_samples= 1) # (B, 1)
            index= torch.cat((index, index_next), dim= 1) #for the new context
            
        return index
    
model= GPTLanguageModel(vocab_size)

m= model.to(device) 



In [12]:
print(torch.cuda.memory_allocated())
print(torch.cuda.memory_reserved())

79955968
83886080


In [13]:
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    print(iter)
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"step: {iter}, train loss: {losses['train']:.3f}, val loss: {losses['val']:.3f}")
    # sample a batch of data
    xb, yb = get_batch('train')
    
    print("Target indices:", yb)

    # evaluate the loss
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
print(loss.item())
    
with open('model-01.pkl', 'wb') as f:
    pickle.dump(model, f)
print('model saved')

0


RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
